In [7]:

# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [8]:
PROJECT_ID = "ninth-stock-473508-c2"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [9]:
BUCKET_URI = f"gs://mlops-course-practice-ninth-stock-473508-c2-unique"  # @param {type:"string"}
MODEL_ARTIFACT_DIR="iris_classifier/model"

In [10]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-course-practice-ninth-stock-473508-c2-unique/...


In [11]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

/opt/conda/lib/python3.10/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [6]:
from feast import FeatureStore
import pandas as pd, joblib, datetime, os
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

store = FeatureStore(repo_path="feature_repo")
df = pd.read_csv("ga_resources/data/raw/iris.csv")
df["flower_id"] = range(1, len(df)+1)
df["event_timestamp"] = pd.Timestamp.now()

# Fetch features for training
features = store.get_historical_features(
    entity_df=df[["flower_id","event_timestamp"]],
    features=[
        "iris_features:sepal_length",
        "iris_features:sepal_width",
        "iris_features:petal_length",
        "iris_features:petal_width",
        "iris_features:species",
    ],
).to_df()

X = features[["sepal_length","sepal_width","petal_length","petal_width"]]
y = features["species"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
model = DecisionTreeClassifier(max_depth=3, random_state=1)
model.fit(X_train, y_train)
acc = accuracy_score(y_test, model.predict(X_test))

print(f"✅ Model trained — Accuracy: {acc:.3f}")

os.makedirs("artifacts", exist_ok=True)
joblib.dump(model, "artifacts/model.joblib")

✅ Model trained — Accuracy: 0.956


['artifacts/model.joblib']

In [12]:
!gsutil cp artifacts/model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://artifacts/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.5 KiB/  2.5 KiB]                                                
Operation completed over 1 objects/2.5 KiB.                                      
